In [15]:
"""
Merge: Painel Event Study + Dados 3G
Prepara dados completos para Callaway & Sant'Anna com controles 3G
"""

import pandas as pd
import numpy as np

In [16]:
# ============================================================================
# 1. CARREGAR DADOS
# ============================================================================

print("\n📊 CARREGANDO DADOS...")

# Painel event study (4G)
painel = pd.read_csv('painel_completo_event_study.csv')

# CRÍTICO: Converter codigo_tse para string
if painel['codigo_tse'].dtype != 'object':
    print(f"  ⚠️  Convertendo codigo_tse de {painel['codigo_tse'].dtype} para string...")
    painel['codigo_tse'] = painel['codigo_tse'].astype(str)

print(f"\nPainel Event Study:")
print(f"  Observações: {len(painel):,}")
print(f"  Municípios: {painel['codigo_tse'].nunique():,}")
print(f"  Anos: {sorted(painel['ano'].unique())}")
print(f"  Colunas: {len(painel.columns)}")
print(f"  Tipo codigo_tse: {painel['codigo_tse'].dtype}")

# Dados 3G
dados_3g = pd.read_csv('municipios_3g_com_codigo_tse.csv')

# CRÍTICO: Converter codigo_tse para string
if dados_3g['codigo_tse'].dtype != 'object':
    print(f"  ⚠️  Convertendo codigo_tse de {dados_3g['codigo_tse'].dtype} para string...")
    dados_3g['codigo_tse'] = dados_3g['codigo_tse'].astype(str)

print(f"\nDados 3G:")
print(f"  Municípios: {len(dados_3g):,}")
print(f"  Colunas: {len(dados_3g.columns)}")
print(f"  Tipo codigo_tse: {dados_3g['codigo_tse'].dtype}")


📊 CARREGANDO DADOS...
  ⚠️  Convertendo codigo_tse de int64 para string...

Painel Event Study:
  Observações: 16,489
  Municípios: 4,123
  Anos: [np.int64(2010), np.int64(2014), np.int64(2018), np.int64(2022)]
  Colunas: 20
  Tipo codigo_tse: object
  ⚠️  Convertendo codigo_tse de int64 para string...

Dados 3G:
  Municípios: 5,524
  Colunas: 19
  Tipo codigo_tse: object


In [17]:
# ============================================================================
# 2. SELECIONAR COLUNAS 3G PARA MERGE
# ============================================================================

print("\n🔧 PREPARANDO MERGE...")

# Colunas 3G que vamos adicionar ao painel
colunas_3g = [
    'codigo_tse',
    'ano_primeira_3g',
    'mes_primeira_3g',
    'n_estacoes_3g_total',
    'tem_3g_2010',
    'tem_3g_2014',
    'tem_3g_2018',
    'tem_3g_2022',
    'anos_com_3g_2014',
    'anos_com_3g_2018',
    'anos_com_3g_2022'
]

# Verificar se todas as colunas existem
colunas_faltando = [col for col in colunas_3g if col not in dados_3g.columns]
if colunas_faltando:
    print(f"\n⚠️  AVISO: Colunas não encontradas em dados_3g: {colunas_faltando}")
    colunas_3g = [col for col in colunas_3g if col in dados_3g.columns]

print(f"\nColunas 3G a serem adicionadas: {len(colunas_3g) - 1}")  # -1 pois codigo_tse é key
for col in colunas_3g[1:]:
    print(f"  - {col}")

# Preparar dados 3G
dados_3g_merge = dados_3g[colunas_3g].copy()



🔧 PREPARANDO MERGE...

Colunas 3G a serem adicionadas: 10
  - ano_primeira_3g
  - mes_primeira_3g
  - n_estacoes_3g_total
  - tem_3g_2010
  - tem_3g_2014
  - tem_3g_2018
  - tem_3g_2022
  - anos_com_3g_2014
  - anos_com_3g_2018
  - anos_com_3g_2022


In [18]:
# ============================================================================
# 3. FAZER MERGE
# ============================================================================

print("\n🔗 EXECUTANDO MERGE...")

# LEFT JOIN: manter todos os municípios do painel
painel_merged = painel.merge(
    dados_3g_merge,
    on='codigo_tse',
    how='left',
    indicator=True
)

# Verificar resultado do merge
print(f"\n RESULTADO DO MERGE:")
print(painel_merged['_merge'].value_counts())
print(f"\n  Total observações: {len(painel_merged):,}")
print(f"  Municípios únicos: {painel_merged['codigo_tse'].nunique():,}")

# Remover coluna _merge
painel_merged = painel_merged.drop('_merge', axis=1)



🔗 EXECUTANDO MERGE...

📊 RESULTADO DO MERGE:
_merge
both          16377
left_only       112
right_only        0
Name: count, dtype: int64

  Total observações: 16,489
  Municípios únicos: 4,123


In [19]:
# ============================================================================
# 4. PREENCHER NAs DOS DADOS 3G
# ============================================================================

print("\n🔧 TRATANDO VALORES AUSENTES...")

# Municípios sem 3G devem ter 0 nas variáveis de cobertura
colunas_preencher_zero = [
    'tem_3g_2010', 'tem_3g_2014', 'tem_3g_2018', 'tem_3g_2022',
    'anos_com_3g_2014', 'anos_com_3g_2018', 'anos_com_3g_2022',
    'n_estacoes_3g_total'
]

for col in colunas_preencher_zero:
    if col in painel_merged.columns:
        antes = painel_merged[col].isna().sum()
        painel_merged[col] = painel_merged[col].fillna(0)
        if antes > 0:
            print(f"  {col}: {antes:,} NaNs → 0")

# Verificar NaNs restantes
print("\n📊 NaNs APÓS TRATAMENTO:")
nans = painel_merged.isna().sum()
nans_relevantes = nans[nans > 0]
if len(nans_relevantes) > 0:
    print(nans_relevantes)
else:
    print("  ✅ Sem NaNs em variáveis críticas!")


🔧 TRATANDO VALORES AUSENTES...
  tem_3g_2010: 112 NaNs → 0
  tem_3g_2014: 112 NaNs → 0
  tem_3g_2018: 112 NaNs → 0
  tem_3g_2022: 112 NaNs → 0
  anos_com_3g_2014: 112 NaNs → 0
  anos_com_3g_2018: 112 NaNs → 0
  anos_com_3g_2022: 112 NaNs → 0
  n_estacoes_3g_total: 112 NaNs → 0

📊 NaNs APÓS TRATAMENTO:
ano_primeira_3g    112
mes_primeira_3g    112
dtype: int64


In [20]:
# ============================================================================
# 5. CRIAR VARIÁVEIS ADICIONAIS PARA CS
# ============================================================================

print("\n🔧 CRIANDO VARIÁVEIS PARA CALLAWAY & SANT'ANNA...")

# Criar variáveis de 3G por ano (para usar como controles no CS)
painel_merged['tem_3g'] = painel_merged.apply(
    lambda row: row[f'tem_3g_{int(row["ano"])}'] if f'tem_3g_{int(row["ano"])}' in painel_merged.columns else 0,
    axis=1
)

painel_merged['anos_3g'] = painel_merged.apply(
    lambda row: row[f'anos_com_3g_{int(row["ano"])}'] if f'anos_com_3g_{int(row["ano"])}' in painel_merged.columns and row['ano'] != 2010 else 0,
    axis=1
)

print("  ✅ tem_3g (por ano)")
print("  ✅ anos_3g (por ano)")



🔧 CRIANDO VARIÁVEIS PARA CALLAWAY & SANT'ANNA...
  ✅ tem_3g (por ano)
  ✅ anos_3g (por ano)


In [21]:
# ============================================================================
# 6. ESTATÍSTICAS DESCRITIVAS
# ============================================================================

print("\n📊 ESTATÍSTICAS DESCRITIVAS DO DATASET FINAL:")

# Por ano
print("\n1. COBERTURA 3G E 4G POR ANO:")
for ano in sorted(painel_merged['ano'].unique()):
    dados_ano = painel_merged[painel_merged['ano'] == ano]
    n = len(dados_ano)
    n_3g = dados_ano['tem_3g'].sum() if 'tem_3g' in dados_ano.columns else 0
    n_4g = dados_ano['tem_4g'].sum() if 'tem_4g' in dados_ano.columns else 0
    
    print(f"\n  {ano}:")
    print(f"    Observações: {n:,}")
    print(f"    Com 3G: {n_3g:,} ({n_3g/n*100:.1f}%)")
    print(f"    Com 4G: {n_4g:,} ({n_4g/n*100:.1f}%)")
    
    # Sobreposição
    if ano in [2018, 2022]:
        ambas = ((dados_ano['tem_3g'] == 1) & (dados_ano['tem_4g'] == 1)).sum()
        so_3g = ((dados_ano['tem_3g'] == 1) & (dados_ano['tem_4g'] == 0)).sum()
        so_4g = ((dados_ano['tem_3g'] == 0) & (dados_ano['tem_4g'] == 1)).sum()
        nenhuma = ((dados_ano['tem_3g'] == 0) & (dados_ano['tem_4g'] == 0)).sum()
        
        print(f"    Sobreposição:")
        print(f"      3G + 4G: {ambas:,} ({ambas/n*100:.1f}%)")
        print(f"      Apenas 3G: {so_3g:,} ({so_3g/n*100:.1f}%)")
        print(f"      Apenas 4G: {so_4g:,} ({so_4g/n*100:.1f}%)")
        print(f"      Nenhuma: {nenhuma:,} ({nenhuma/n*100:.1f}%)")

# Polarização
print(f"\n2. POLARIZAÇÃO (ESTEBAN-RAY):")
print(f"    Média geral: {painel_merged['esteban_ray'].mean():.4f}")
print(f"    Desvio padrão: {painel_merged['esteban_ray'].std():.4f}")
print(f"    Mínimo: {painel_merged['esteban_ray'].min():.4f}")
print(f"    Máximo: {painel_merged['esteban_ray'].max():.4f}")


📊 ESTATÍSTICAS DESCRITIVAS DO DATASET FINAL:

1. COBERTURA 3G E 4G POR ANO:

  2010:
    Observações: 4,120
    Com 3G: 3,475.0 (84.3%)
    Com 4G: 0 (0.0%)

  2014:
    Observações: 4,123
    Com 3G: 3,614.0 (87.7%)
    Com 4G: 879 (21.3%)

  2018:
    Observações: 4,123
    Com 3G: 3,957.0 (96.0%)
    Com 4G: 2,420 (58.7%)
    Sobreposição:
      3G + 4G: 2,419 (58.7%)
      Apenas 3G: 1,538 (37.3%)
      Apenas 4G: 1 (0.0%)
      Nenhuma: 165 (4.0%)

  2022:
    Observações: 4,123
    Com 3G: 4,094.0 (99.3%)
    Com 4G: 4,058 (98.4%)
    Sobreposição:
      3G + 4G: 4,031 (97.8%)
      Apenas 3G: 63 (1.5%)
      Apenas 4G: 27 (0.7%)
      Nenhuma: 2 (0.0%)

2. POLARIZAÇÃO (ESTEBAN-RAY):
    Média geral: 0.0746
    Desvio padrão: 0.0325
    Mínimo: 0.0096
    Máximo: 0.1506


In [22]:
# ============================================================================
# 7. VERIFICAÇÕES DE QUALIDADE
# ============================================================================

print("\n🔍 VERIFICAÇÕES DE QUALIDADE:")

# 1. Verificar se não há duplicatas
duplicatas = painel_merged.duplicated(subset=['codigo_tse', 'ano']).sum()
print(f"\n  1. Duplicatas (codigo_tse + ano): {duplicatas}")
if duplicatas > 0:
    print("     ⚠️  PROBLEMA: Há observações duplicadas!")
else:
    print("     ✅ OK")

# 2. Verificar se anos_com_3g fazem sentido
if 'anos_com_3g_2018' in painel_merged.columns and 'anos_com_3g_2014' in painel_merged.columns:
    dados_2018 = painel_merged[painel_merged['ano'] == 2018].copy()
    problema = dados_2018['anos_com_3g_2018'] < dados_2018['anos_com_3g_2014']
    print(f"\n  2. Anos de 3G crescentes: {len(dados_2018) - problema.sum()} / {len(dados_2018)}")
    if problema.sum() > 0:
        print(f"     ⚠️  {problema.sum()} observações com anos decrescentes")
    else:
        print("     ✅ OK")

# 3. Verificar lógica: se tem 4G, também tem 3G?
for ano in [2018, 2022]:
    if ano in painel_merged['ano'].values:
        dados_ano = painel_merged[painel_merged['ano'] == ano]
        tem_4g_sem_3g = ((dados_ano['tem_4g'] == 1) & (dados_ano['tem_3g'] == 0)).sum()
        total_4g = dados_ano['tem_4g'].sum()
        print(f"\n  3. Ano {ano}: 4G sem 3G: {tem_4g_sem_3g} / {total_4g:.0f}")
        if tem_4g_sem_3g > 0:
            pct = tem_4g_sem_3g / total_4g * 100
            if pct > 10:
                print(f"     ⚠️  {pct:.1f}% dos municípios com 4G não têm 3G (esperado!)")
            else:
                print(f"     ℹ️  {pct:.1f}% (pode acontecer em alguns casos)")
        else:
            print("     ✅ Todos com 4G também têm 3G")

# 4. Verificar cohorts
print(f"\n  4. Cohorts 4G únicos: {painel_merged['cohort'].nunique()}")
print(f"     Cohorts: {sorted(painel_merged['cohort'].unique())}")



🔍 VERIFICAÇÕES DE QUALIDADE:

  1. Duplicatas (codigo_tse + ano): 0
     ✅ OK

  2. Anos de 3G crescentes: 4123 / 4123
     ✅ OK

  3. Ano 2018: 4G sem 3G: 1 / 2420
     ℹ️  0.0% (pode acontecer em alguns casos)

  3. Ano 2022: 4G sem 3G: 27 / 4058
     ℹ️  0.7% (pode acontecer em alguns casos)

  4. Cohorts 4G únicos: 4
     Cohorts: ['cohort_2010_2012', 'cohort_2012_2014', 'cohort_2014_2018', 'cohort_2018_2022']


In [23]:
# ============================================================================
# 8. SALVAR ARQUIVO FINAL
# ============================================================================

print("\n💾 SALVANDO ARQUIVO FINAL...")

# Salvar
output_file = 'painel_event_study_com_3g.csv'
painel_merged.to_csv(output_file, index=False)

print(f"\n✅ Arquivo salvo: {output_file}")
print(f"   Observações: {len(painel_merged):,}")
print(f"   Municípios: {painel_merged['codigo_tse'].nunique():,}")
print(f"   Colunas: {len(painel_merged.columns)}")

# Listar colunas adicionadas
colunas_originais = set(painel.columns)
colunas_novas = set(painel_merged.columns) - colunas_originais
if colunas_novas:
    print(f"\n   Colunas adicionadas ({len(colunas_novas)}):")
    for col in sorted(colunas_novas):
        print(f"     - {col}")



💾 SALVANDO ARQUIVO FINAL...

✅ Arquivo salvo: painel_event_study_com_3g.csv
   Observações: 16,489
   Municípios: 4,123
   Colunas: 32

   Colunas adicionadas (12):
     - ano_primeira_3g
     - anos_3g
     - anos_com_3g_2014
     - anos_com_3g_2018
     - anos_com_3g_2022
     - mes_primeira_3g
     - n_estacoes_3g_total
     - tem_3g
     - tem_3g_2010
     - tem_3g_2014
     - tem_3g_2018
     - tem_3g_2022


In [24]:
# ============================================================================
# 9. RESUMO FINAL
# ============================================================================

print("\n" + "="*70)
print("✅ MERGE CONCLUÍDO COM SUCESSO!")
print("="*70)

print("\n📋 PRÓXIMOS PASSOS:")
print("\n  1. Usar arquivo: painel_event_study_com_3g.csv")
print("\n  2. Executar CS_completo.Rmd no RStudio:")
print("     - Alterar linha de carregamento:")
print('       dados <- fread("painel_event_study_com_3g.csv")')
print("\n  3. Clicar Knit e aguardar resultados!")

print("\n📊 VARIÁVEIS DISPONÍVEIS PARA CS:")
print("   - yname: esteban_ray")
print("   - gname: ano_primeira_4g")
print("   - tname: ano")
print("   - xformla: ~ tem_3g + anos_3g  (CONTROLES)")

print("\n" + "="*70)



✅ MERGE CONCLUÍDO COM SUCESSO!

📋 PRÓXIMOS PASSOS:

  1. Usar arquivo: painel_event_study_com_3g.csv

  2. Executar CS_completo.Rmd no RStudio:
     - Alterar linha de carregamento:
       dados <- fread("painel_event_study_com_3g.csv")

  3. Clicar Knit e aguardar resultados!

📊 VARIÁVEIS DISPONÍVEIS PARA CS:
   - yname: esteban_ray
   - gname: ano_primeira_4g
   - tname: ano
   - xformla: ~ tem_3g + anos_3g  (CONTROLES)



In [27]:
# ============================================
# MERGE: DADOS 3G CORRETOS → PAINEL CS
# ============================================

import pandas as pd
import numpy as np

# Carregar painel atual
painel = pd.read_csv('painel_event_study_com_3g.csv')

# Carregar dados 3G CORRETOS
dados_3g_correto = pd.read_csv('municipios_3g_com_codigo_tse.csv')

print(f"📊 Painel atual: {len(painel)} observações, {painel['codigo_tse'].nunique()} municípios")
print(f"📊 Dados 3G: {len(dados_3g_correto)} municípios")

# Converter codigo_tse para string (consistência)
painel['codigo_tse'] = painel['codigo_tse'].astype(str)
dados_3g_correto['codigo_tse'] = dados_3g_correto['codigo_tse'].astype(str)

# Remover colunas 3G antigas (INCORRETAS) do painel
colunas_3g_antigas = [col for col in painel.columns if '3g' in col.lower()]
print(f"\n🗑️  Removendo colunas 3G antigas: {colunas_3g_antigas}")
painel = painel.drop(columns=colunas_3g_antigas)

# Merge com dados 3G CORRETOS
painel_corrigido = painel.merge(
    dados_3g_correto[[
        'codigo_tse',
        'primeira_estacao_3g',
        'ano_primeira_3g',
        'mes_primeira_3g',
        'n_estacoes_3g_total',
        'tem_3g_2010',
        'tem_3g_2014',
        'tem_3g_2018',
        'tem_3g_2022',
        'anos_com_3g_2014',
        'anos_com_3g_2018',
        'anos_com_3g_2022'
    ]],
    on='codigo_tse',
    how='left'
)

# Verificar merge
print(f"\n📊 Painel corrigido: {len(painel_corrigido)} observações")
print(f"📊 Municípios sem dados 3G: {painel_corrigido['ano_primeira_3g'].isna().sum() / 4}")  # /4 porque 4 anos

# Criar variáveis dinâmicas tem_3g e anos_3g (CORRETAS)
painel_corrigido['tem_3g'] = np.where(
    painel_corrigido['ano'] == 2010,
    painel_corrigido['tem_3g_2010'],
    np.where(
        painel_corrigido['ano'] == 2014,
        painel_corrigido['tem_3g_2014'],
        np.where(
            painel_corrigido['ano'] == 2018,
            painel_corrigido['tem_3g_2018'],
            painel_corrigido['tem_3g_2022']
        )
    )
)

painel_corrigido['anos_3g'] = np.where(
    painel_corrigido['ano'] == 2014,
    painel_corrigido['anos_com_3g_2014'],
    np.where(
        painel_corrigido['ano'] == 2018,
        painel_corrigido['anos_com_3g_2018'],
        painel_corrigido['anos_com_3g_2022']
    )
)

# Preencher NaN com 0 (municípios sem 3G)
painel_corrigido['tem_3g'] = painel_corrigido['tem_3g'].fillna(0)
painel_corrigido['anos_3g'] = painel_corrigido['anos_3g'].fillna(0)

# ============================================
# COMPARAÇÃO: ANTIGA vs NOVA
# ============================================

print("\n" + "="*70)
print("COMPARAÇÃO: COBERTURA 3G ANTIGA (ERRADA) vs NOVA (CORRETA)")
print("="*70)

# Você precisará carregar o arquivo antigo para comparar
painel_antigo = pd.read_csv('painel_event_study_com_3g.csv')

comparacao = pd.DataFrame({
    'ano': [2010, 2014, 2018, 2022],
    'pct_3g_ANTIGA': [
        painel_antigo[painel_antigo['ano']==2010]['tem_3g'].mean() * 100,
        painel_antigo[painel_antigo['ano']==2014]['tem_3g'].mean() * 100,
        painel_antigo[painel_antigo['ano']==2018]['tem_3g'].mean() * 100,
        painel_antigo[painel_antigo['ano']==2022]['tem_3g'].mean() * 100
    ],
    'pct_3g_NOVA': [
        painel_corrigido[painel_corrigido['ano']==2010]['tem_3g'].mean() * 100,
        painel_corrigido[painel_corrigido['ano']==2014]['tem_3g'].mean() * 100,
        painel_corrigido[painel_corrigido['ano']==2018]['tem_3g'].mean() * 100,
        painel_corrigido[painel_corrigido['ano']==2022]['tem_3g'].mean() * 100
    ],
    'anos_3g_ANTIGA': [
        0,  # Não havia em 2010 na versão antiga
        painel_antigo[painel_antigo['ano']==2014]['anos_3g'].mean(),
        painel_antigo[painel_antigo['ano']==2018]['anos_3g'].mean(),
        painel_antigo[painel_antigo['ano']==2022]['anos_3g'].mean()
    ],
    'anos_3g_NOVA': [
        0,  # Não calculamos para 2010
        painel_corrigido[painel_corrigido['ano']==2014]['anos_3g'].mean(),
        painel_corrigido[painel_corrigido['ano']==2018]['anos_3g'].mean(),
        painel_corrigido[painel_corrigido['ano']==2022]['anos_3g'].mean()
    ]
})

comparacao['diff_pct'] = comparacao['pct_3g_NOVA'] - comparacao['pct_3g_ANTIGA']
comparacao['diff_anos'] = comparacao['anos_3g_NOVA'] - comparacao['anos_3g_ANTIGA']

print("\n", comparacao.to_string(index=False))

# Salvar
painel_corrigido.to_csv(
    'painel_event_study_com_3g_final.csv',
    index=False
)

print(f"\n✅ Painel CORRIGIDO salvo: painel_event_study_com_3g_final.csv")


📊 Painel atual: 16489 observações, 4123 municípios
📊 Dados 3G: 5524 municípios

🗑️  Removendo colunas 3G antigas: ['ano_primeira_3g', 'mes_primeira_3g', 'n_estacoes_3g_total', 'tem_3g_2010', 'tem_3g_2014', 'tem_3g_2018', 'tem_3g_2022', 'anos_com_3g_2014', 'anos_com_3g_2018', 'anos_com_3g_2022', 'tem_3g', 'anos_3g']

📊 Painel corrigido: 16489 observações
📊 Municípios sem dados 3G: 28.0

COMPARAÇÃO: COBERTURA 3G ANTIGA (ERRADA) vs NOVA (CORRETA)

  ano  pct_3g_ANTIGA  pct_3g_NOVA  anos_3g_ANTIGA  anos_3g_NOVA  diff_pct  diff_anos
2010      84.344660    84.344660        0.000000      0.000000       0.0        0.0
2014      87.654620    87.654620        9.114902      9.114902       0.0        0.0
2018      95.973805    95.973805       12.826041     12.826041       0.0        0.0
2022      99.296629    99.296629       16.731916     16.731916       0.0        0.0

✅ Painel CORRIGIDO salvo: painel_event_study_com_3g_final.csv
